In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy scipy scikit-learn pyyaml
!nvidia-smi
!pip install torch==2.4.1+cu124 torchvision==0.19.1+cu124 torchaudio==2.4.1+cu124 --index-url https://download.pytorch.org/whl/cu124
import torch
print(torch.__version__)
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html
import dgl

/bin/bash: line 1: nvidia-smi: command not found
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 659.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 167.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 171.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 931.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 MB 5.9 MB/s eta 0:00:00
   

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 98.8 MB/s eta 0:00:00


In [ ]:
import json

file_path = '/content/drive/MyDrive/GE-GNN/MY_work/DataSets/Sports_and_Outdoors.json'
object_count = 0
try:
    with open(file_path, 'r') as f:
        for line in f:
            # Assuming each line is a valid JSON object
            try:
                json.loads(line)
                object_count += 1
            except json.JSONDecodeError:
                # Handle lines that are not valid JSON if necessary
                pass
    print(f"The file contains {object_count} objects.")
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

The file contains 3013256 objects.


In [ ]:
# Block 1: Data Loading, Filtering, and Low-Dimensional Feature Engineering

from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import category_encoders as ce
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import warnings
import pickle
import os

warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Mount Google Drive
drive.mount('/content/drive')

# Load the initial dataset
file_path = '/content/drive/MyDrive/GE-GNN/MY_work/DataSets/Sports_and_Outdoors.json'
df = pd.read_json(file_path, lines=True)

# Filter dataset: specified samples per class
df_class_0 = df[df['class'] == 0]
df_class_1 = df[df['class'] == 1]

random_state = 42
n_samples_class_0 = 20000  # Frauds
n_samples_class_1 = 80000  # Non-frauds

df_class_0_sampled = df_class_0.sample(n=min(len(df_class_0), n_samples_class_0), random_state=random_state, replace=False)
df_class_1_sampled = df_class_1.sample(n=min(len(df_class_1), n_samples_class_1), random_state=random_state, replace=False)


df_filtered_csv = pd.concat([df_class_0_sampled, df_class_1_sampled]).reset_index(drop=True)

# Save filtered dataset
output_file_path_csv = '/content/drive/MyDrive/GE-GNN/MY_work/DataSets/filtered_sports_and_outdoors_100k_fraud_20k_nonfraud_80k.csv'
df_filtered_csv.to_csv(output_file_path_csv, index=False)
print(f"Filtered dataset saved: {output_file_path_csv}")

# ---------------------- Feature Engineering ----------------------

# Helpful votes
df_filtered_csv['helpful_votes'] = df_filtered_csv['helpful'].apply(lambda x: x[0])
df_filtered_csv['unhelpful_votes'] = df_filtered_csv['helpful'].apply(lambda x: x[1])
df_filtered_csv['total_votes'] = df_filtered_csv['helpful_votes'] + df_filtered_csv['unhelpful_votes']

df_filtered_csv['helpful_unhelpful_ratio'] = df_filtered_csv.apply(
    lambda row: row['helpful_votes'] / row['total_votes'] if row['total_votes'] != 0 else 0,
    axis=1
)

# Review time gap features
df_filtered_csv['reviewTime_dt'] = pd.to_datetime(df_filtered_csv['reviewTime'], format='%m %d, %Y')
df_filtered_csv_sorted = df_filtered_csv.sort_values(by=['reviewerID', 'reviewTime_dt'])
df_filtered_csv_sorted['prev_reviewTime_dt'] = df_filtered_csv_sorted.groupby('reviewerID')['reviewTime_dt'].shift(1)
df_filtered_csv_sorted['day_gap'] = (df_filtered_csv_sorted['reviewTime_dt'] - df_filtered_csv_sorted['prev_reviewTime_dt']).dt.days.fillna(0)

df_filtered_csv = df_filtered_csv.loc[df_filtered_csv_sorted.index].copy()
df_filtered_csv['day_gap'] = df_filtered_csv_sorted['day_gap']
df_filtered_csv['same_day_indicator'] = (df_filtered_csv['day_gap'] == 0).astype(int)

df_filtered_csv['total_helpful_votes'] = df_filtered_csv.groupby('reviewerID')['helpful_votes'].transform('sum')
df_filtered_csv['total_unhelpful_votes'] = df_filtered_csv.groupby('reviewerID')['unhelpful_votes'].transform('sum')

df_filtered_csv['review_word_count'] = df_filtered_csv['reviewText'].apply(lambda x: len(str(x).split()))

# Sentiment scores
try:
    nltk.data.find('sentiment/vader_lexicon.zip')
except LookupError:
    nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()
df_filtered_csv['sentiment_score'] = df_filtered_csv['reviewText'].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])

df_filtered_csv['reviewText'] = df_filtered_csv['reviewText'].fillna('')
df_filtered_csv['summary'] = df_filtered_csv['summary'].fillna('')
df_filtered_csv['reviewerName'] = df_filtered_csv['reviewerName'].fillna('')

# TF-IDF + SVD for review text
tfidf_review_text = TfidfVectorizer(max_features=5000)
review_text_embeddings = tfidf_review_text.fit_transform(df_filtered_csv['reviewText'])
svd_review_text = TruncatedSVD(n_components=7, random_state=42)
review_text_svd = svd_review_text.fit_transform(review_text_embeddings)
review_text_svd_df = pd.DataFrame(review_text_svd, columns=[f'review_text_svd_{i}' for i in range(review_text_svd.shape[1])])

# TF-IDF + SVD for summary
tfidf_summary = TfidfVectorizer(max_features=2000)
summary_embeddings = tfidf_summary.fit_transform(df_filtered_csv['summary'])
svd_summary = TruncatedSVD(n_components=3, random_state=42)
summary_svd = svd_summary.fit_transform(summary_embeddings)
summary_svd_df = pd.DataFrame(summary_svd, columns=[f'summary_svd_{i}' for i in range(summary_svd.shape[1])])

# ✅ Frequency Encoding (NO leakage)
freq_encoder_reviewerID = ce.CountEncoder(cols=['reviewerID'])
reviewerID_encoded = freq_encoder_reviewerID.fit_transform(df_filtered_csv['reviewerID'])
reviewerID_encoded.columns = ['reviewerID_encoded']

freq_encoder_asin = ce.CountEncoder(cols=['asin'])
asin_encoded = freq_encoder_asin.fit_transform(df_filtered_csv['asin'])
asin_encoded.columns = ['asin_encoded']

freq_encoder_reviewerName = ce.CountEncoder(cols=['reviewerName'])
reviewerName_encoded = freq_encoder_reviewerName.fit_transform(df_filtered_csv['reviewerName'])
reviewerName_encoded.columns = ['reviewerName_encoded']

numerical_features_selected = df_filtered_csv[['overall', 'helpful_votes', 'unhelpful_votes',
                                               'helpful_unhelpful_ratio', 'day_gap', 'same_day_indicator',
                                               'review_word_count', 'sentiment_score',
                                               'total_helpful_votes', 'total_unhelpful_votes', 'total_votes']]

# Combine all low-dim features
engineered_features_df_low_dim = pd.concat([
    numerical_features_selected.reset_index(drop=True),
    review_text_svd_df.reset_index(drop=True),
    summary_svd_df.reset_index(drop=True),
    reviewerID_encoded.reset_index(drop=True),
    asin_encoded.reset_index(drop=True),
    reviewerName_encoded.reset_index(drop=True)
], axis=1)

print("\nLow-dimensional features generated:")
display(engineered_features_df_low_dim.head())
print(f"Total Features: {engineered_features_df_low_dim.shape[1]}")
print("\n✅ Block-1 completed — No label leakage now!")

# ================================================================
# ✅ SAVE ALL ENCODERS FOR FUTURE INFERENCE (VERY IMPORTANT)
# ================================================================

ENCODER_DIR = "/content/drive/MyDrive/GE-GNN/encoders"
os.makedirs(ENCODER_DIR, exist_ok=True)

print("\n✅ Saving encoders used during training...")

# TF-IDF + SVD encoders
with open(f"{ENCODER_DIR}/tfidf_review.pkl", "wb") as f:
    pickle.dump(tfidf_review_text, f)

with open(f"{ENCODER_DIR}/svd_review.pkl", "wb") as f:
    pickle.dump(svd_review_text, f)

with open(f"{ENCODER_DIR}/tfidf_summary.pkl", "wb") as f:
    pickle.dump(tfidf_summary, f)

with open(f"{ENCODER_DIR}/svd_summary.pkl", "wb") as f:
    pickle.dump(svd_summary, f)

# Frequency Encoders
with open(f"{ENCODER_DIR}/freq_reviewerID.pkl", "wb") as f:
    pickle.dump(freq_encoder_reviewerID, f)

with open(f"{ENCODER_DIR}/freq_asin.pkl", "wb") as f:
    pickle.dump(freq_encoder_asin, f)

with open(f"{ENCODER_DIR}/freq_reviewerName.pkl", "wb") as f:
    pickle.dump(freq_encoder_reviewerName, f)

print("✅ Encoders saved to:", ENCODER_DIR)
print("✅ These must be loaded during inference — do NOT re-fit encoders!")
# ================================================================

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Filtered dataset saved: /content/drive/MyDrive/GE-GNN/MY_work/DataSets/filtered_sports_and_outdoors_100k_fraud_20k_nonfraud_80k.csv


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...



Low-dimensional features generated:


,overall,helpful_votes,unhelpful_votes,helpful_unhelpful_ratio,day_gap,same_day_indicator,review_word_count,sentiment_score,total_helpful_votes,total_unhelpful_votes,...,review_text_svd_3,review_text_svd_4,review_text_svd_5,review_text_svd_6,summary_svd_0,summary_svd_1,summary_svd_2,reviewerID_encoded,asin_encoded,reviewerName_encoded
0,1,0,1,0.0,0.0,1,22,-0.4019,0,1,...,-0.021572,-0.097558,-0.007729,0.071458,0.029516,0.215615,-0.034107,1,2,2
1,3,0,0,0.0,0.0,1,51,0.9224,0,0,...,0.001261,-0.042690,0.090665,0.033525,0.055534,0.143786,0.193735,1,2,12
2,5,1,1,0.5,0.0,1,20,0.9652,1,1,...,0.116697,-0.105428,-0.147906,-0.059715,0.182732,0.229841,-0.067016,1,1,33
3,5,0,0,0.0,0.0,1,26,0.0000,0,0,...,0.043062,0.145209,-0.028022,-0.047291,0.538921,-0.118240,0.096294,1,5,1
4,3,1,1,0.5,0.0,1,49,0.6297,1,1,...,-0.064168,-0.073333,-0.029173,0.005126,0.004431,0.003045,0.000511,1,1,1


Total Features: 24

✅ Block-1 completed — No label leakage now!

✅ Saving encoders used during training...
✅ Encoders saved to: /content/drive/MyDrive/GE-GNN/encoders
✅ These must be loaded during inference — do NOT re-fit encoders!


In [ ]:
# This cell is now empty and can be safely deleted.

In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.io
import pandas as pd

# Ensure cell EQfQraB2NpGz (Data Loading, Filtering, and Low-Dimensional Feature Engineering)
# has been executed successfully before running this cell to define
# df_filtered_csv and engineered_features_df_low_dim.

# Assuming df_filtered_csv and engineered_features_df_low_dim are available from previous steps.
# If not, reload them:
# file_path = '/content/drive/MyDrive/GE-GNN/MY_work/DataSets/filtered_sports_and_outdoors.csv'
# df_filtered_csv = pd.read_csv(file_path)
# # Reapply feature engineering steps if needed to get engineered_features_df_low_dim
# from sklearn.decomposition import TruncatedSVD
# from sklearn.feature_extraction.text import TfidfVectorizer
# import category_encoders as ce
# ... (reapply feature engineering code)

# 1. Extract features and labels
features = engineered_features_df_low_dim.values.astype(np.float32)
labels = df_filtered_csv['class'].values.astype(np.int64)

# Reshape labels to be a 1xN array similar to Amazon.mat
labels = labels.reshape(1, -1)

# 2. Create mappings for reviewerID, asin, and review index
# The review index is simply the DataFrame index after filtering and sampling
review_index_map = {original_idx: new_idx for new_idx, original_idx in enumerate(df_filtered_csv.index)}
num_nodes = len(df_filtered_csv) # Number of nodes is the number of reviews

# 3. Initialize lists to store edge indices for each relationship type
upu_src = []
upu_dst = []
usu_src = []
usu_dst = []
uvu_src = []
uvu_dst = []

# Group reviews by product (asin) to find U-P-U relationships
for asin, group in df_filtered_csv.groupby('asin'):
    review_indices = group.index.tolist()
    # Create edges between all pairs of reviews for the same product
    for i in range(len(review_indices)):
        for j in range(len(review_indices)):
            if i != j:
                # Use the new node indices from the review_index_map
                upu_src.append(review_index_map[review_indices[i]])
                upu_dst.append(review_index_map[review_indices[j]])

# Group reviews by reviewer (reviewerID) to find U-S-U and U-V-U relationships
for reviewer_id, group in df_filtered_csv.groupby('reviewerID'):
    review_indices = group.index.tolist()
    # Iterate through pairs of reviews by the same reviewer
    for i in range(len(review_indices)):
        for j in range(len(review_indices)):
            if i != j:
                review1_idx = review_indices[i]
                review2_idx = review_indices[j]

                # Check for U-S-U relationship (same sentiment - simplifying to same overall rating)
                if df_filtered_csv.loc[review1_idx, 'overall'] == df_filtered_csv.loc[review2_idx, 'overall']:
                    usu_src.append(review_index_map[review1_idx])
                    usu_dst.append(review_index_map[review2_idx])

                # Check for U-V-U relationship (similar review length - simplifying to within a threshold, e.g., 10 words difference)
                if abs(df_filtered_csv.loc[review1_idx, 'review_word_count'] - df_filtered_csv.loc[review2_idx, 'review_word_count']) <= 10:
                    uvu_src.append(review_index_map[review1_idx])
                    uvu_dst.append(review_index_map[review2_idx])


# 4. Create sparse adjacency matrices for each relationship type
upu_adj_matrix = sp.coo_matrix((np.ones(len(upu_src), dtype=np.float32), (upu_src, upu_dst)), shape=(num_nodes, num_nodes))
usu_adj_matrix = sp.coo_matrix((np.ones(len(usu_src), dtype=np.float32), (usu_src, usu_dst)), shape=(num_nodes, num_nodes))
uvu_adj_matrix = sp.coo_matrix((np.ones(len(uvu_src), dtype=np.float32), (uvu_src, uvu_dst)), shape=(num_nodes, num_nodes))

# 5. Define the path where the output .mat file will be saved.
output_mat_file_path_hetero = '/content/drive/MyDrive/GE-GNN/Mat_Dgl_files/amazon_like_sports_outdoors_hetero_20kf_pavan.mat'

# 6. Create a dictionary similar to Amazon.mat with heterogeneous graph structure
amazon_like_data_hetero = {
    '__header__': b'Custom generated MAT-file (Heterogeneous Graph)',
    '__version__': '1.0',
    '__globals__': [],
    'features': features,
    'label': labels,
    'net_upu': upu_adj_matrix, # U-P-U relationship
    'net_usu': usu_adj_matrix, # U-S-U relationship
    'net_uvu': uvu_adj_matrix  # U-V-U relationship
}

# 7. Save the dictionary to a .mat file
scipy.io.savemat(output_mat_file_path_hetero, amazon_like_data_hetero)

# 8. Print confirmation and shapes
print(f"Processed heterogeneous data saved to {output_mat_file_path_hetero}")
print(f"Shape of features matrix: {amazon_like_data_hetero['features'].shape}")
print(f"Shape of label matrix: {amazon_like_data_hetero['label'].shape}")
print(f"Shape of net_upu adjacency matrix: {amazon_like_data_hetero['net_upu'].shape}")
print(f"Shape of net_usu adjacency matrix: {amazon_like_data_hetero['net_usu'].shape}")
print(f"Shape of net_uvu adjacency matrix: {amazon_like_data_hetero['net_uvu'].shape}")
print(f"Number of net_upu edges: {amazon_like_data_hetero['net_upu'].nnz}")
print(f"Number of net_usu edges: {amazon_like_data_hetero['net_usu'].nnz}")
print(f"Number of net_uvu edges: {amazon_like_data_hetero['net_uvu'].nnz}")

In [ ]:
import sys
import torch
import dgl
import scipy.io as scio
import numpy as np
from sklearn.model_selection import train_test_split

# Define the path to the generated heterogeneous .mat file and the output DGL graph file
mat_file_path_hetero = '/content/drive/MyDrive/GE-GNN/Mat_Dgl_files/amazon_like_sports_outdoors_hetero_20kf_pavan.mat'
output_dgl_path_hetero = '/content/drive/MyDrive/GE-GNN/Mat_Dgl_files/sports_outdoors_hetero_pavan_20k_F.dgl'

# Load the data from the generated .mat file
try:
    mat_data_hetero = scio.loadmat(mat_file_path_hetero)
    print("Generated heterogeneous .mat file loaded successfully!")
except FileNotFoundError:
    print(f"Error: The generated .mat file was not found at {mat_file_path_hetero}")
    sys.exit()
except Exception as e:
    print(f"An error occurred while loading the .mat file: {e}")
    sys.exit()

# Extract features, labels, and adjacency matrices for each relationship type
features = torch.from_numpy(mat_data_hetero['features']).float()
labels = torch.from_numpy(mat_data_hetero['label'][0]).long() # Assuming labels are in a 1xN array

# Get sparse matrices for each relationship
upu_adj_matrix = mat_data_hetero['net_upu']
usu_adj_matrix = mat_data_hetero['net_usu']
uvu_adj_matrix = mat_data_hetero['net_uvu']

# Convert sparse matrices to source and destination node lists for DGL
upu_src = torch.from_numpy(upu_adj_matrix.row).int()
upu_dst = torch.from_numpy(upu_adj_matrix.col).int()

usu_src = torch.from_numpy(usu_adj_matrix.row).int()
usu_dst = torch.from_numpy(usu_adj_matrix.col).int()

uvu_src = torch.from_numpy(uvu_adj_matrix.row).int()
uvu_dst = torch.from_numpy(uvu_adj_matrix.col).int()


# Define the graph structure for DGL (heterogeneous graph with a single node type 'r' for review)
# Node type 'r' (review) and edge types 'p' (product), 's' (sentiment), 'v' (review length)
graph_structure_hetero = {
    ('r', 'p', 'r'): (upu_src, upu_dst), # Reviews connected if for same Product
    ('r', 's', 'r'): (usu_src, usu_dst), # Reviews connected if by same User with same Sentiment
    ('r', 'v', 'r'): (uvu_src, uvu_dst)  # Reviews connected if by same User with similar review Length
}

# The number of nodes should be the number of reviews, which is the number of rows in the features matrix
num_nodes = features.shape[0]

# Create the DGL heterogeneous graph with the correct number of nodes
sports_outdoors_hetero_graph = dgl.heterograph(graph_structure_hetero, num_nodes_dict={'r': num_nodes})

# Add features and labels to the nodes
sports_outdoors_hetero_graph.nodes['r'].data['feat'] = features
sports_outdoors_hetero_graph.nodes['r'].data['label'] = labels

# Optional: Add self-loops for each edge type if needed (based on original script)
for etype in sports_outdoors_hetero_graph.etypes:
    sports_outdoors_hetero_graph.add_self_loop(etype=etype)

# Generate dataset partition (train, validation, test masks)
index = list(range(num_nodes))
dataset_l = num_nodes

# Using the same random_state as the original filtering step for reproducibility
random_state = 42
train_ratio = 0.4 # Example ratios, adjust as needed
test_ratio = 0.67 # Example ratios, adjust as needed

# Stratified split based on labels
train_idx, rest_idx, train_lbs, rest_lbs = train_test_split(
    index, labels.numpy(), stratify=labels.numpy(), train_size=train_ratio, random_state=random_state, shuffle=True
)
valid_idx, test_idx, _, _ = train_test_split(
    rest_idx, rest_lbs, stratify=rest_lbs, test_size=test_ratio, random_state=random_state, shuffle=True
)

# Create masks
train_mask = torch.zeros(dataset_l, dtype=torch.bool)
train_mask[np.array(train_idx)] = True
valid_mask = torch.zeros(dataset_l, dtype=torch.bool)
valid_mask[np.array(valid_idx)] = True
test_mask = torch.zeros(dataset_l, dtype=torch.bool)
test_mask[np.array(test_idx)] = True

# Add masks to the graph
sports_outdoors_hetero_graph.nodes['r'].data['trn_msk'] = train_mask
sports_outdoors_hetero_graph.nodes['r'].data['val_msk'] = valid_mask
sports_outdoors_hetero_graph.nodes['r'].data['tst_msk'] = test_mask

# Save the DGL graph
dgl.save_graphs(output_dgl_path_hetero, sports_outdoors_hetero_graph)

print(f"DGL heterogeneous graph saved to {output_dgl_path_hetero}")

In [ ]:
# ge_gnn_train.py
# Full, corrected, ready-to-run script for the GE-GNN pipeline.
# Expects a YAML config like the one you provided (dataset, seed, epoch, lr, cuda, ...)

import argparse
import os
import sys
import copy
import random
import yaml
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dgl
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, recall_score

# ----------------------
# Utility functions
# ----------------------
def setup_seed(seed: int):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def normalize_rows(mx: np.ndarray):
    # row-normalize dense matrix
    rowsum = np.array(mx.sum(1)) + 1e-8
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    return (r_inv[:, None] * mx).astype(np.float32)

def conf_gmean(conf):
    # conf is 2x2 [ [tn, fp], [fn, tp] ]
    tn, fp, fn, tp = conf.ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    return float(np.sqrt(tpr * tnr))

def evaluate(labels, logits, result_path=''):
    # logits: torch.Tensor (N, n_class)
    probs = F.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()
    preds = logits.argmax(1).detach().cpu().numpy()
    if len(result_path) > 0:
        np.save(result_path + '_result_preds.npy', preds)
        np.save(result_path + '_result_probs.npy', probs)
    conf = confusion_matrix(labels, preds)
    recall = recall_score(labels, preds, zero_division=0)
    f1_macro = f1_score(labels, preds, average='macro', zero_division=0)
    auc = roc_auc_score(labels, probs) if len(np.unique(labels)) > 1 else 0.5
    gmean = conf_gmean(conf)
    return f1_macro, auc, gmean, recall

# ----------------------
# Model components
# ----------------------
class RelationAware(nn.Module):
    def __init__(self, input_dim, output_dim, dropout):
        super().__init__()
        self.d_linear = nn.Linear(input_dim, output_dim)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(dropout)
    def forward(self, src, dst):
        src = self.d_linear(src)
        dst = self.d_linear(dst)
        diff = src - dst
        edge_sum = self.tanh(src + dst + diff)
        return edge_sum

class H_layer(nn.Module):
    def __init__(self, input_dim, output_dim, head, relation_aware, etype, dropout, if_sum=False):
        super().__init__()
        self.etype = etype
        self.head = head
        self.hd = output_dim
        self.if_sum = if_sum
        self.atten = nn.Linear(3 * self.hd, 1)
        self.relu = nn.ReLU()
        self.relation_aware = relation_aware
        self.leakyrelu = nn.LeakyReLU()
        self.softmax = nn.Softmax(dim=1)
        self.w_linear = nn.Linear(input_dim, output_dim * head)

    def forward(self, g, h):
        # Works with both homograph and heterograph as in your original code (you used g.ndata)
        with g.local_scope():
            # Ensure 'feat' data is accessible within the local scope for this layer
            if 'r' in g.ntypes and 'feat' in g.nodes['r'].data:
                g.ndata['feat'] = g.nodes['r'].data['feat']
            elif 'feat' in g.ndata:
                 g.ndata['feat'] = g.ndata['feat']
            else:
                 raise RuntimeError("Graph missing 'feat' in ndata or nodes['r'].data for H_layer.")

            # apply edge function on the given etype
            try:
                g.apply_edges(self.sign_edges, etype=self.etype)
                edge_sum = g.edges[self.etype].data['edge_sum']
            except Exception:
                # fallback to global edata (if using homogeneous)
                g.apply_edges(self.sign_edges)
                edge_sum = g.edata['edge_sum']

            h_proj = self.w_linear(h)
            g.ndata['h'] = h_proj
            try:
                g.update_all(self.message, self.reduce, etype=self.etype)
            except Exception:
                g.update_all(self.message, self.reduce)

            out = g.ndata['out']
            edge_s = g.ndata['s']
            if not self.if_sum:
                return edge_s, out, h_proj.view(-1, self.head * self.hd)
            else:
                return edge_s, out, h_proj.view(-1, self.head, self.hd).sum(-2)

    def message(self, edges):
        src_f = edges.src['h'].view(-1, self.head, self.hd)
        dst_f = edges.dst['h'].view(-1, self.head, self.hd)
        edge_s = edges.data['edge_sum'].view(-1, self.head, self.hd)
        z = torch.cat([src_f, dst_f, edge_s], dim=-1)  # (E, head, 3*hd)
        alpha = self.atten(z)  # (E, head, 1)
        alpha = self.leakyrelu(alpha)
        return {'atten': alpha, 'sf': src_f, 'edge_s': edge_s}

    def reduce(self, nodes):
        alpha = nodes.mailbox['atten']  # (N, deg, head, 1)
        sf = nodes.mailbox['sf']        # (N, deg, head, hd)
        alpha = self.softmax(alpha)
        out = torch.sum(alpha * sf, dim=1)  # (N, head, hd)
        if not self.if_sum:
            out = out.view(-1, self.head * self.hd)
            edge_s = torch.mean(nodes.mailbox['edge_s'], dim=1).view(-1, self.head * self.hd)
            return {'out': out, 's': edge_s}
        else:
            out = out.sum(dim=-2)
            edge_s = torch.sum(torch.mean(nodes.mailbox['edge_s'], dim=1), dim=-2)
            return {'out': out, 's': edge_s}


    def sign_edges(self, edges):
        src = edges.src['feat']
        dst = edges.dst['feat']
        edge_sum = self.relation_aware(src, dst)
        return {'edge_sum': edge_sum}


class Gate(nn.Module):
    def __init__(self, head, output_dim, dropout, if_sum=False):
        super().__init__()
        self.output_dim = output_dim
        self.head = head
        if not if_sum:
            self.beta = nn.Parameter(torch.empty(size=(2 * self.head * self.output_dim, 1)))
            nn.init.xavier_normal_(self.beta.data, gain=1.414)
        else:
            self.beta = nn.Parameter(torch.empty(size=(2 * self.output_dim, 1)))
            nn.init.xavier_normal_(self.beta.data, gain=1.414)
        self.sigmoid = nn.Sigmoid()

    def forward(self, edge_sum, out, h):
        beta = torch.cat([edge_sum, out], dim=1)
        gate = self.sigmoid(torch.matmul(beta, self.beta))
        final = gate * out + (1 - gate) * h
        return final

class MultiRelationGE_GNNLayer(nn.Module):
    def __init__(self, input_dim, output_dim, head, dataset, dropout, if_sum=False):
        super().__init__()
        # dataset here is the DGLGraph object or wrapper; original code used dataset.etypes
        self.relation = copy.deepcopy(dataset.etypes) if hasattr(dataset, 'etypes') else []
        if 'homo' in self.relation:
            self.relation.remove('homo')
        self.n_relation = len(self.relation)
        self.if_sum = if_sum
        if not self.if_sum:
            self.liner = nn.Linear(max(1, self.n_relation) * output_dim * head, output_dim * head)
        else:
            self.liner = nn.Linear(max(1, self.n_relation) * output_dim, output_dim)
        self.relation_aware = RelationAware(input_dim, output_dim * head, dropout)

        self.minelayers = nn.ModuleDict()
        self.dropout = nn.Dropout(dropout)
        for e in (self.relation if len(self.relation) > 0 else ['homo']):
            sub = nn.ModuleList()
            sub.append(H_layer(input_dim, output_dim, head, self.relation_aware, e, dropout, if_sum))
            sub.append(Gate(head, output_dim, dropout, if_sum))
            self.minelayers[e] = sub

    def forward(self, g, h):
        hs = []
        for e, modules in self.minelayers.items():
            edge_sum1, out1, h1 = modules[0](g, h)
            he = modules[1](edge_sum1, out1, h1)
            hs.append(he)
        x = torch.cat(hs, dim=1)
        x = self.dropout(x)
        x = self.liner(x)
        return x

    # Note: The loss method was part of the MultiRelationGE_GNNLayer in the original code,
    # but it seems it should be in the main GE_GNN model for the training loop to work correctly.
    # If the original code's loss calculation logic is needed, it should be adapted or moved
    # to a training loop that utilizes the forward pass and calculates loss based on model outputs and labels.
    # For now, I will provide a basic loss calculation based on the model's forward pass
    # and available training masks/labels, assuming the required data is in g.ndata or g.nodes['r'].data.

    def loss(self, g):
        # Compute training loss.
        # This loss function structure was copied from the original code's MultiRelationGE_GNNLayer.
        # To fully replicate the original training, this logic should likely be adapted or moved
        # to a training loop that utilizes the forward pass and calculates loss based on model outputs and labels.
        # For now, I will provide a basic loss calculation based on the model's forward pass
        # and available training masks/labels, assuming the required data is in g.ndata or g.nodes['r'].data.

        # Get training mask and labels, handling heterogeneous graphs
        if 'r' in g.ntypes and 'trn_msk' in g.nodes['r'].data: # Corrected mask name
             train_mask = g.nodes['r'].data['trn_msk'].bool()
             labels = g.nodes['r'].data['label']
        elif 'train_mask' in g.ndata:
            train_mask = g.ndata['train_mask'].bool()
            labels = g.ndata['label']
        else:
             print("Error: Graph missing 'trn_msk' node data for loss calculation.")
             return torch.tensor(0.0, device=g.device)

        # Ensure there are training samples
        if not train_mask.any():
            print("Warning: No nodes in the training mask. Skipping loss calculation.")
            return torch.tensor(0.0, device=g.device)

        train_label = labels[train_mask]

        # Perform forward pass to get logits
        logits = self.forward(g)[train_mask]

        # Calculate cross-entropy loss
        model_loss = F.cross_entropy(logits, train_label)

        # Note: Prototype loss from original code is not implemented here as it was in MultiRelationGE_GNNLayer.
        # If needed, this method should be updated to reflect the correct loss calculation logic
        # based on the original GE-GNN implementation, potentially involving prototype calculations.
        # For now, returning only the cross-entropy loss.
        # print("Warning: Prototype loss calculation is not fully implemented in the loss method.")

        return model_loss


class GE_GNN(nn.Module):
    def __init__(self, args, g):
        super().__init__()
        self.n_layer = args.n_layer
        # Determine input_dim based on graph features, prioritizing heterogeneous node data
        if hasattr(g, 'ntypes') and 'r' in g.ntypes and 'feat' in g.nodes['r'].data: # Check for 'feat' in heterogeneous graph
            self.input_dim = g.nodes['r'].data['feat'].shape[1]
        elif 'feature' in g.ndata: # Fallback to homogeneous graph 'feature'
            self.input_dim = g.ndata['feature'].shape[1]
        else:
            # Fallback or raise error if features are not found
            print("Warning: 'feat' not found in nodes['r'].data and 'feature' not found in ndata. Assuming input_dim from config if available.")
            self.input_dim = getattr(args, 'input_dim', 0) # Use config if available, default to 0
            if self.input_dim == 0:
                 raise RuntimeError("Graph does not contain 'feat' node data (heterogeneous) or 'feature' node data (homogeneous) and 'input_dim' not specified in config.")


        self.intra_dim = args.intra_dim
        self.n_class = args.n_class
        self.gamma1 = args.gamma1
        self.gamma2 = args.gamma2
        self.mine_layers = nn.ModuleList()

        if args.n_layer == 1:
            self.mine_layers.append(MultiRelationGE_GNNLayer(self.input_dim, self.n_class, args.head, g, args.dropout, if_sum=True))
        else:
            self.mine_layers.append(MultiRelationGE_GNNLayer(self.input_dim, self.intra_dim, args.head, g, args.dropout))
            for _ in range(1, args.n_layer - 1):
                # Input dimension for subsequent layers is the output dimension of the previous layer
                prev_layer_output_dim = self.intra_dim * args.head # Assuming MultiRelationGE_GNNLayer outputs concatenated heads
                self.mine_layers.append(MultiRelationGE_GNNLayer(prev_layer_output_dim, self.intra_dim, args.head, g, args.dropout))
            # Final layer sums the heads
            prev_layer_output_dim = self.intra_dim * args.head # Output of the last intermediate layer
            self.mine_layers.append(MultiRelationGE_GNNLayer(prev_layer_output_dim, self.n_class, args.head, g, args.dropout, if_sum=True))


        self.dropout = nn.Dropout(args.dropout)
        self.relu = nn.ReLU()

    def forward(self, g):
        # Get features from the graph, handling heterogeneous graphs
        if 'r' in g.ntypes and 'feat' in g.nodes['r'].data: # Prioritize heterogeneous graph 'feat'
             feats = g.nodes['r'].data['feat'].float()
        elif 'feature' in g.ndata: # Fallback to homogeneous graph 'feature'
            feats = g.ndata['feature'].float()
        else:
            raise RuntimeError("Graph missing 'feat' in nodes['r'].data or 'feature' in ndata for forward pass.")


        h = feats # Start with initial features as input to the first layer
        for i, layer in enumerate(self.mine_layers):
            h = layer(g, h)
            if i < len(self.mine_layers) - 1: # Apply ReLU and Dropout for intermediate layers
                h = self.relu(h)
                h = self.dropout(h)
        return h

    def loss(self, g):
        # Compute training loss.
        # This loss function structure was copied from the original code's MultiRelationGE_GNNLayer.
        # To fully replicate the original training, this logic should likely be adapted or moved
        # to a training loop that utilizes the forward pass and calculates loss based on model outputs and labels.
        # For now, I will provide a basic loss calculation based on the model's forward pass
        # and available training masks/labels, assuming the required data is in g.ndata or g.nodes['r'].data.

        # Get training mask and labels, handling heterogeneous graphs
        if 'r' in g.ntypes and 'trn_msk' in g.nodes['r'].data: # Corrected mask name
             train_mask = g.nodes['r'].data['trn_msk'].bool()
             labels = g.nodes['r'].data['label']
        elif 'train_mask' in g.ndata:
            train_mask = g.ndata['train_mask'].bool()
            labels = g.ndata['label']
        else:
             print("Error: Graph missing 'trn_msk' node data for loss calculation.")
             return torch.tensor(0.0, device=g.device)

        # Ensure there are training samples
        if not train_mask.any():
            print("Warning: No nodes in the training mask. Skipping loss calculation.")
            return torch.tensor(0.0, device=g.device)

        train_label = labels[train_mask]

        # Perform forward pass to get logits
        logits = self.forward(g)[train_mask]

        # Calculate cross-entropy loss
        model_loss = F.cross_entropy(logits, train_label)

        # Note: Prototype loss from original code is not implemented here as it was in MultiRelationGE_GNNLayer.
        # If needed, this method should be updated to reflect the correct loss calculation logic
        # based on the original GE-GNN implementation, potentially involving prototype calculations.
        # For now, returning only the cross-entropy loss.
        # print("Warning: Prototype loss calculation is not fully implemented in the loss method.")

        return model_loss


# ----------------------
# Training script
# ----------------------
def load_config(yaml_path):
    with open(yaml_path, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

def train_graph(args, g, model_path_base, dataset_name):
    """Trains the GE-GNN model on a single DGL graph."""
    # build model
    model = GE_GNN(args, g).to(args.device)

    # optimizer and scheduler
    optimizer = optim.Adam(model.parameters(), lr=float(args.lr), weight_decay=float(args.weight_decay))
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20, verbose=True)

    best_auc = -1.0
    best_epoch = 0

    # gradient clip
    max_grad_norm = 5.0

    print(f"Start training model for {dataset_name}...")
    for e in range(int(args.epoch)):
        model.train()
        optimizer.zero_grad()
        # Calculate loss using the model's loss method
        loss = model.loss(g)
        loss.backward()
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()

        # validation
        model.eval()
        with torch.no_grad():
            # Ensure valid_mask exists in g.ndata or g.nodes['r'].data
            if 'r' in g.ntypes and 'val_msk' in g.nodes['r'].data: # Corrected mask name
                 valid_mask = g.nodes['r'].data['val_msk'].bool()
                 valid_labels = g.nodes['r'].data['label'][valid_mask].cpu().numpy()
                 logits = model(g)[valid_mask]
            elif 'valid_mask' in g.ndata:
                valid_mask = g.ndata['valid_mask'].bool()
                valid_labels = g.ndata['label'][valid_mask].cpu().numpy()
                logits = model(g)[valid_mask]
            else:
                 print("Warning: 'val_msk' not found in graph. Skipping validation.")
                 continue # Skip validation if mask is missing

            if len(valid_labels) == 0:
                 print("Warning: No validation samples found. Skipping validation metrics.")
                 auc = 0.0 # Assign a default low AUC
                 f1_macro, gmean, recall = 0.0, 0.0, 0.0
            else:
                f1_macro, auc, gmean, recall = evaluate(valid_labels, logits)

        if args.log:
            print(f"Dataset: {dataset_name}, Epoch: {e}: Loss:{loss.item():.4f}, Valid AUC:{auc:.6f}, Recall:{recall:.4f}, F1:{f1_macro:.4f}, G-Mean:{gmean:.4f}")

        # scheduler step by validation AUC
        scheduler.step(auc)

        # early save
        if auc > best_auc:
            best_auc = auc
            best_epoch = e
            # Save model with dataset name in filename
            current_model_path = os.path.join(args.result_path, f"20k_fraud_model_head{args.head}.pt")
            torch.save(model.state_dict(), current_model_path)
            if args.log:
                print(f"Saved best model for {dataset_name} at epoch {e}, best_auc={best_auc:.6f}")

        # early stopping
        # simple patience tracking using early_stop from config
        if (e - best_epoch) >= int(args.early_stop):
            if args.log:
                print(f"Dataset: {dataset_name}: No improvement for {args.early_stop} epochs. Early stopping at epoch {e}.")
            break

    print(f"End training for {dataset_name}")

    # test
    final_model_path = os.path.join(args.result_path, f"20k_fraud_model_head{args.head}.pt")
    if os.path.exists(final_model_path):
        model.load_state_dict(torch.load(final_model_path, map_location=args.device))
        model.eval()
        with torch.no_grad():
            # Ensure test_mask exists in g.ndata or g.nodes['r'].data
             if 'r' in g.ntypes and 'tst_msk' in g.nodes['r'].data: # Corrected mask name
                 test_mask = g.nodes['r'].data['tst_msk'].bool()
                 test_labels = g.nodes['r'].data['label'][test_mask].cpu().numpy()
                 logits = model(g)[test_mask].cpu()
             elif 'test_mask' in g.ndata:
                test_mask = g.ndata['test_mask'].bool()
                test_labels = g.ndata['label'][test_mask].cpu().numpy()
                logits = model(g)[test_mask].cpu()
             else:
                 print(f"Warning: 'tst_msk' not found in graph for {dataset_name}. Skipping test evaluation.")
                 return # Skip test if mask is missing

             if len(test_labels) == 0:
                 print(f"Warning: No test samples found for {dataset_name}. Skipping test metrics.")
             else:
                f1_macro, auc, gmean, recall = evaluate(test_labels, logits, os.path.join(args.result_path, dataset_name))
                print(f"Test Results for {dataset_name}: F1-macro:{f1_macro:.6f}, AUC:{auc:.6f}, G-Mean:{gmean:.6f}, Recall:{recall:.6f}")
    else:
        print(f"No saved model found at {final_model_path}; skipping test for {dataset_name}.")

    return g # Return the processed graph


def main(config_path):
    """Main function to load config and run training."""
    cfg = load_config(config_path)
    args = argparse.Namespace(**cfg)

    setup_seed(args.seed)

    args.device = torch.device('cuda' if args.cuda and torch.cuda.is_available() else 'cpu')

    # --- Load the specified DGL graph ---
    dgl_graph_path = "/content/drive/MyDrive/GE-GNN/Mat_Dgl_files/sports_outdoors_hetero_pavan_20k_F.dgl"
    try:
        graphs, labels_dict = dgl.load_graphs(dgl_graph_path)
        g = graphs[0] # Assuming the DGL file contains a list of graphs, and we want the first one
        print(f"Loaded DGL graph from: {dgl_graph_path}")

        # Ensure graph and node data are on the correct device
        g = g.to(args.device)
        if 'r' in g.ntypes:
            for key in g.nodes['r'].data:
                g.nodes['r'].data[key] = g.nodes['r'].data[key].to(args.device)
        # If it's a homogeneous graph
        else:
             for key in g.ndata:
                g.ndata[key] = g.ndata[key].to(args.device)


    except FileNotFoundError:
        print(f"Error: DGL graph file not found at {dgl_graph_path}")
        sys.exit()
    except Exception as e:
        print(f"An error occurred while loading the DGL graph: {e}")
        sys.exit()
    # ------------------------------------


    # Set dataset_name for output file naming
    # The dataset_name variable was previously used in the filename.
    # Now we will hardcode "20k_fraud" into the filename logic below.
    # dataset_name = os.path.splitext(os.path.basename(dgl_graph_path))[0]
    dataset_name = "sports_outdoors_20k_F" # Set dataset_name explicitly for consistent naming

    # Create results directory if it doesn't exist
    if not os.path.exists(args.result_path):
        os.makedirs(args.result_path)

    # Train the model, passing a relevant name for the output file if needed by train_graph
    # We will pass a simplified name since the specific "20k_fraud" is now hardcoded in train_graph
    processed_graph = train_graph(args, g, args.result_path, dataset_name)
    return processed_graph


if __name__ == "__main__":
    # Change this path if necessary; points to your uploaded YAML file
    config_path = "/content/drive/MyDrive/GE-GNN/config/amazon.yaml"
    # Assign the returned graph to the global variable 'g'
    g = main(config_path)

In [ ]:
config_path = "/content/drive/MyDrive/GE-GNN/config/amazon.yaml"
import argparse
import os
import sys
import dgl
import torch
import numpy as np
import scipy.io as scio
from sklearn.model_selection import train_test_split
import torch.optim as optim
import yaml
import random
import scipy.sparse as sp
import torch.nn.functional as F
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, recall_score
import copy
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore')
def setup_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def parse_arg():
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str, default='amazon')
    args = parser.parse_args()
    config_path = './config/'+args.dataset+'.yaml'
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    args = argparse.Namespace(**config)
    print('----------------------------------')
    print('              args')
    print('----------------------------------')
    print(f'dataset:\t{args.dataset}')
    print(f'seed:\t{args.seed}')
    print(f'epoch:\t{args.epoch}')
    print(f'early_stop:\t{args.early_stop}')
    print(f'lr:\t{args.lr}')
    print(f'weigth_decay:{args.weight_decay}')
    print(f'gamma1:\t{args.gamma1}')
    print(f'gamma2:\t{args.gamma2}')
    print(f'intra_dim:\t{args.intra_dim}')
    print(f'head:\t{args.head}')
    print(f'n_layer:\t{args.n_layer}')
    print(f'dropout:\t{args.dropout}')
    print(f'cuda:\t{args.cuda}')
    print('----------------------------------')
    return args
# args = parse_arg() # Comment out or remove this line to avoid parsing command line arguments in Colab


class EarlyStop():
    def __init__(self, early_stop, if_more=True) -> None:
        self.best_eval = 0
        self.best_epoch = 0
        self.if_more = if_more
        self.early_stop = early_stop
        self.stop_steps = 0

    def step(self, current_eval, current_epoch):
        do_stop = False
        do_store = False
        if self.if_more:
            if current_eval > self.best_eval:
                self.best_eval = current_eval
                self.best_epoch = current_epoch
                self.stop_steps = 1
                do_store = True
            else:
                self.stop_steps += 1
                if self.stop_steps >= self.early_stop:
                    do_stop = True
        else:
            if current_eval < self.best_eval:
                self.best_eval = current_eval
                self.best_epoch = current_epoch
                self.stop_steps = 1
                do_store = True
            else:
                self.stop_steps += 1
                if self.stop_steps >= self.early_stop:
                    do_stop = True
        return do_store, do_stop

def conf_gmean(conf):
        tn, fp, fn, tp = conf.ravel()
        # Avoid division by zero if denominator is zero
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        return (tpr*tnr)**0.5

def prob2pred(prob, threshhold=0.5):
    pred = np.zeros_like(prob, dtype=np.int32)
    pred[prob >= threshhold] = 1
    pred[prob < threshhold] = 0
    return pred

def evaluate(labels, logits, result_path = ''):
    probs = F.softmax(logits, dim=1)[:,1].cpu().numpy()
    preds = logits.argmax(1).cpu().numpy()
    if len(result_path)>0:
        np.save(result_path+'_result_preds', preds)
        np.save(result_path+'_result_probs'
                , probs)
    conf = confusion_matrix(labels, preds)
    recall = recall_score(labels, preds, zero_division=0) # Added zero_division
    f1_macro = f1_score(labels, preds, average='macro', zero_division=0) # Added zero_division
    auc = roc_auc_score(labels, probs) if len(np.unique(labels)) > 1 else 0.5 # Added check for unique labels
    gmean = conf_gmean(conf)
    return f1_macro, auc, gmean, recall

def hinge_loss(labels, scores):
    margin = 1
    ls = labels*scores

    loss = F.relu(margin-ls)
    loss = loss.mean()
    return loss

def normalize(mx):
        """
                Row-normalize sparse matrix
                Code from https://github.com/williamleif/graphsage-simple/
        """
        rowsum = np.array(mx.sum(1)) + 0.01
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = sp.diags(r_inv)
        mx = r_mat_inv.dot(mx)
        return mx
class H_layer(nn.Module):
    def __init__(self, input_dim, output_dim, head, relation_aware, etype, dropout, if_sum=False):
        super().__init__()
        self.etype = etype
        self.head = head
        self.hd = output_dim
        self.if_sum = if_sum
        self.atten = nn.Linear(3*self.hd, 1)
        self.relu = nn.ReLU()
        self.relation_ware = relation_aware
        self.leakyrelu = nn.LeakyReLU()
        self.softmax = nn.Softmax(dim=1)
        self.w_liner = nn.Linear(input_dim, output_dim*head)


    def forward(self, g, h):
        with g.local_scope():
            # Use 'feat' for heterogeneous graph node data
            if 'r' in g.ntypes:
                g.nodes['r'].data['feat'] = h
            else: # Fallback for homogeneous graph if needed
                g.ndata['feat'] = h

            # Apply edge function on the given etype, specifying etype for heterogeneous graphs
            g.apply_edges(self.sign_edges, etype=self.etype)
            edge_sum = g.edges[self.etype].data['edge_sum']

            h = self.w_liner(h)
            # Use 'h' for node data in update_all
            if 'r' in g.ntypes:
                g.nodes['r'].data['h'] = h
            else: # Fallback for homogeneous graph
                g.ndata['h'] = h

            # Update all, specifying etype for heterogeneous graphs
            g.update_all(message_func=self.message, reduce_func=self.reduce, etype=self.etype)

            # Retrieve output and edge_sum from node data
            if 'r' in g.ntypes:
                out = g.nodes['r'].data['out']
                edge_s = g.nodes['r'].data['s']
            else: # Fallback for homogeneous graph
                out = g.ndata['out']
                edge_s = g.ndata['s']


            if not self.if_sum:
                return edge_s, out, h.view(-1, self.head*self.hd)
            else:
                return edge_s, out, h.view(-1, self.head, self.hd).sum(-2)


    def message(self, edges):
        src_f = edges.src['h']
        src_f = src_f.view(-1, self.head, self.hd)
        dst_f = edges.dst['h']
        dst_f = dst_f.view(-1, self.head, self.hd)
        edge_s = edges.data['edge_sum'].view(-1, self.head, self.hd)
        z = torch.cat([src_f, dst_f, edge_s], dim=-1)

        alpha = self.atten(z)
        alpha = self.leakyrelu(alpha)
        return {'atten':alpha, 'sf':src_f, 'edge_s': edge_s}


    def reduce(self, nodes):
        alpha = nodes.mailbox['atten']
        sf = nodes.mailbox['sf']
        alpha = self.softmax(alpha)
        out = torch.sum(alpha*sf, dim=1)
        if not self.if_sum:
            out = out.view(-1, self.head*self.hd)
            edge_s = torch.mean(nodes.mailbox['edge_s'], dim=1).view(-1, self.head*self.hd)
            return {'out':out, 's': edge_s}
        else:
            out = out.sum(dim=-2)
            edge_s = torch.sum(torch.mean(nodes.mailbox['edge_s'], dim=1), dim=-2)

            return {'out':out, 's': edge_s}


    def sign_edges(self, edges):
        # Access features from the edges object directly
        src = edges.src['feat']
        dst = edges.dst['feat']

        edge_sum = self.relation_ware(src, dst)
        return {'edge_sum':edge_sum}

class Gate(nn.Module):
    def __init__(self, head, output_dim, dropout, if_sum=False):
        super().__init__()
        self.output_dim = output_dim
        self.head = head
        if not if_sum:
            self.beta = nn.Parameter(torch.empty(size=(2*self.head*self.output_dim, 1)))
            nn.init.xavier_normal_(self.beta.data, gain=1.414)
        else:

            self.beta = nn.Parameter(torch.empty(size=(2*self.output_dim, 1)))
            nn.init.xavier_normal_(self.beta.data, gain=1.414)
        self.sigmoid = nn.Sigmoid()

    def forward(self, edge_sum, out, h):
        beta = torch.cat([edge_sum, out], dim=1)
        gate = self.sigmoid(torch.matmul(beta, self.beta))
        final = gate * out + (1 - gate) * h
        return final


class RelationAware(nn.Module):
    def __init__(self, input_dim, output_dim, dropout):
        super().__init__()
        self.d_liner = nn.Linear(input_dim, output_dim)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(dropout)
    def forward(self, src, dst):
        src = self.d_liner(src)
        dst = self.d_liner(dst)
        diff = src-dst
        edge_sum = self.tanh(src + dst + diff)
        return edge_sum


class MultiRelationGE_GNNLayer(nn.Module):
    def __init__(self, input_dim, output_dim, head, dataset, dropout, if_sum=False):
        super().__init__()
        self.relation = copy.deepcopy(dataset.etypes)
        if 'homo' in self.relation:
            self.relation.remove('homo')
        self.n_relation = len(self.relation)
        self.if_sum = if_sum
        if not self.if_sum:
            self.liner = nn.Linear(self.n_relation*output_dim*head, output_dim*head)
        else:
            self.liner = nn.Linear(self.n_relation*output_dim, output_dim)
        self.relation_aware = RelationAware(input_dim, output_dim*head, dropout)

        self.minelayers = nn.ModuleDict()
        self.sublayer = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)


#         if not self.if_sum:
        for e in self.relation:
            self.sublayer = nn.ModuleList()
            self.minelayers[e] = self.sublayer
            self.sublayer.append(H_layer(input_dim, output_dim, head, self.relation_aware, e, dropout, if_sum))
            self.sublayer.append(Gate(head, output_dim, dropout, if_sum))

    def forward(self, g, h):
        hs = []
        for e in self.relation:
            edge_sum1, out1, h1 = self.minelayers[e][0](g, h)
            he = self.minelayers[e][1](edge_sum1, out1, h1)
            hs.append(he)
        x = torch.cat(hs, dim=1)
        x = self.dropout(x)
        x = self.liner(x)
        return x

    def loss(self, g, h):
        with g.local_scope():
            # Use 'feat' for heterogeneous graph node data
            if 'r' in g.ntypes:
                g.nodes['r'].data['feat'] = h
            else: # Fallback for homogeneous graph
                g.ndata['feat'] = h

            # Use correct mask names for heterogeneous graph
            if 'r' in g.ntypes:
                train_mask = g.nodes['r'].data['trn_msk'].bool()
                labels = g.nodes['r'].data['label']
            else: # Fallback for homogeneous graph
                train_mask = g.ndata['train_mask'].bool()
                labels = g.ndata['label']


            train_label = labels[train_mask]
            train_pos = train_label==1
            train_neg = train_label==0
            train_pos_index = train_pos.nonzero().flatten().detach().cpu().numpy()
            train_neg_index = train_neg.nonzero().flatten().detach().cpu().numpy()
            # Handle case where there might be fewer negative samples than positive
            size_neg_sample = min(len(train_pos_index), len(train_neg_index))
            train_neg_index = np.random.choice(train_neg_index, size=size_neg_sample, replace=False)

            node_index = np.concatenate([train_neg_index, train_pos_index])
            np.random.shuffle(node_index) # Shuffle the indices

            hs = []
            diff_loss = 0
            for e in self.relation:
                edge_sum1, out1, h1 = self.minelayers[e][0](g, h)
                he = self.minelayers[e][1](edge_sum1, out1, h1)
                hs.append(he)

#                 diff_loss += diff_loss1
            x = torch.cat(hs, dim=1)
            x = self.dropout(x)
            agg_h = self.liner(x)
#             diff_loss1 = F.cross_entropy(agg_h[train_mask][node_index], train_label[node_index])
            if not self.if_sum:
                # Ensure indices are applied correctly after masking
                masked_agg_h = agg_h[train_mask]
                valid_indices = node_index[node_index < masked_agg_h.shape[0]]
                diff_loss1 = F.cross_entropy(masked_agg_h[valid_indices], train_label[valid_indices])
            else:
                # Ensure indices are applied correctly after masking
                masked_agg_h = agg_h[train_mask]
                valid_indices = node_index[node_index < masked_agg_h.shape[0]]
                diff_loss1 = F.cross_entropy(masked_agg_h[valid_indices], train_label[valid_indices])
#             diff_loss2 = F.cross_entropy(self.filter2(h[train_mask][node_index]), train_label[node_index])
#             diff_loss3 = F.cross_entropy(h[train_mask][node_index], train_label[node_index])
            return agg_h, diff_loss1



class GE_GNN(nn.Module):
    def __init__(self, args, g):
        super().__init__()
        self.n_layer = args.n_layer
        # Determine input_dim based on graph features, prioritizing heterogeneous node data
        if hasattr(g, 'ntypes') and 'r' in g.ntypes and 'feat' in g.nodes['r'].data: # Check for 'feat' in heterogeneous graph
            self.input_dim = g.nodes['r'].data['feat'].shape[1]
        elif 'feature' in g.ndata: # Fallback to homogeneous graph 'feature'
            self.input_dim = g.ndata['feature'].shape[1]
        else:
            # Fallback or raise error if features are not found
            print("Warning: 'feat' not found in nodes['r'].data and 'feature' not found in ndata. Assuming input_dim from config if available.")
            self.input_dim = getattr(args, 'input_dim', 0) # Use config if available, default to 0
            if self.input_dim == 0:
                 raise RuntimeError("Graph does not contain 'feat' node data (heterogeneous) or 'feature' node data (homogeneous) and 'input_dim' not specified in config.")


        self.intra_dim = args.intra_dim
        self.n_class = args.n_class
        self.gamma1 = args.gamma1
        self.gamma2 = args.gamma2
        self.n_layer = args.n_layer
        self.mine_layers = nn.ModuleList()
        if args.n_layer == 1:
            self.mine_layers.append(MultiRelationGE_GNNLayer(self.input_dim, self.n_class, args.head, g, args.dropout, if_sum=True))
        else:
            self.mine_layers.append(MultiRelationGE_GNNLayer(self.input_dim, self.intra_dim, args.head, g, args.dropout))
            for _ in range(1, self.n_layer-1):
                self.mine_layers.append(MultiRelationGE_GNNLayer(self.intra_dim*args.head, self.intra_dim, args.head, g, args.dropout))
            self.mine_layers.append(MultiRelationGE_GNNLayer(self.intra_dim*args.head, self.n_class, args.head, g, args.dropout, if_sum=True))
        self.dropout = nn.Dropout(args.dropout)
        self.relu = nn.ReLU()

    def forward(self, g):
        # Get features from the graph, handling heterogeneous graphs
        if 'r' in g.ntypes and 'feat' in g.nodes['r'].data: # Prioritize heterogeneous graph 'feat'
             feats = g.nodes['r'].data['feat'].float()
        elif 'feature' in g.ndata: # Fallback to homogeneous graph 'feature'
            feats = g.ndata['feature'].float()
        else:
            raise RuntimeError("Graph missing 'feat' in nodes['r'].data or 'feature' in ndata for forward pass.")

        h = feats # Start with initial features as input to the first layer
        for i, layer in enumerate(self.mine_layers):
            h = layer(g, h)
            if i < len(self.mine_layers) - 1: # Apply ReLU and Dropout for intermediate layers
                h = self.relu(h)
                h = self.dropout(h)
        return h

    def loss(self, g):
        # Compute training loss.
        # This loss function structure was copied from the original code's MultiRelationGE_GNNLayer.
        # To fully replicate the original training, this logic should likely be adapted or moved
        # to a training loop that utilizes the forward pass and calculates loss based on model outputs and labels.
        # For now, I will provide a basic loss calculation based on the model's forward pass
        # and available training masks/labels, assuming the required data is in g.ndata or g.nodes['r'].data.

        # Get features from the graph, handling heterogeneous graphs
        if 'r' in g.ntypes and 'feat' in g.nodes['r'].data: # Prioritize heterogeneous graph 'feat'
             feats = g.nodes['r'].data['feat'].float()
        elif 'feature' in g.ndata: # Fallback to homogeneous graph 'feature'
            feats = g.ndata['feature'].float()
        else:
            raise RuntimeError("Graph missing 'feat' in nodes['r'].data or 'feature' in ndata for loss calculation.")


        # Use correct mask names for heterogeneous graph
        if 'r' in g.ntypes and 'trn_msk' in g.nodes['r'].data:
             train_mask = g.nodes['r'].data['trn_msk'].bool()
             labels = g.nodes['r'].data['label']
        elif 'train_mask' in g.ndata:
            train_mask = g.ndata['train_mask'].bool()
            labels = g.ndata['label']
        else:
             print("Error: Graph missing 'trn_msk' node data for loss calculation.")
             return torch.tensor(0.0, device=g.device)


        train_label = labels[train_mask]
        train_pos = train_label == 1
        train_neg = train_label == 0

        pos_index = train_pos.nonzero().flatten().detach().cpu().numpy()
        neg_index = train_neg.nonzero().flatten().detach().cpu().numpy()
        # Handle case where there might be fewer negative samples than positive
        size_neg_sample = min(len(pos_index), len(neg_index))
        neg_index = np.random.choice(neg_index, size=size_neg_sample, replace=False)

        index = np.concatenate([pos_index, neg_index])
        np.random.shuffle(index) # Shuffle the indices

        h = feats # Start with initial features as input to the first layer
        prototype_loss = 0
        for i, layer in enumerate(self.mine_layers):
            # Need to pass the graph 'g' and the current hidden state 'h' to the layer's loss method
            # The layer's loss method also returns the updated hidden state 'h'
            if hasattr(layer, 'loss'): # Check if the layer has a loss method (MultiRelationGE_GNNLayer does)
                 h, p_loss = layer.loss(g, h) # Pass graph and current hidden state
                 prototype_loss += p_loss
            else:
                # If the layer doesn't have a specific loss method, just perform forward pass
                h = layer(g, h)

            if i < len(self.mine_layers) - 1: # Apply ReLU and Dropout for intermediate layers
                h = self.relu(h)
                h = self.dropout(h)


        # Ensure indices are applied correctly after masking
        masked_h = h[train_mask]
        valid_indices = index[index < masked_h.shape[0]] # Filter indices to be within masked tensor bounds
        model_loss = F.cross_entropy(masked_h[valid_indices], train_label[valid_indices])

        # Note: The original code had '1.2 * prototype_loss / 3'. Adjust multiplier as needed based on the intended weighting.
        # Assuming the sum of prototype_loss from each layer is desired.
        loss = model_loss + 1.2 * prototype_loss # Adjust multiplier as needed

        return loss

# Load args from YAML
config_path = "/content/drive/MyDrive/GE-GNN/config/amazon.yaml" # Use the specified config path
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
args = argparse.Namespace(**config) # Create args namespace from config

setup_seed(args.seed)
device = torch.device(args.cuda if torch.cuda.is_available() and args.cuda else 'cpu')
args.device = device

# --- Load the specified DGL graph ---
dgl_graph_path = "/content/drive/MyDrive/GE-GNN/Mat_Dgl_files/sports_outdoors_hetero_pavan_20k_F.dgl"
try:
    graphs, labels_dict = dgl.load_graphs(dgl_graph_path)
    g = graphs[0] # Assuming the DGL file contains a list of graphs, and we want the first one
    print(f"Loaded DGL graph from: {dgl_graph_path}")

    # Ensure graph and node data are on the correct device
    g = g.to(args.device)
    if 'r' in g.ntypes:
        for key in g.nodes['r'].data:
            g.nodes['r'].data[key] = g.nodes['r'].data[key].to(args.device)
    # If it's a homogeneous graph (fallback, though we expect heterogeneous)
    else:
         for key in g.ndata:
            g.ndata[key] = g.ndata[key].to(args.device)


except FileNotFoundError:
    print(f"Error: DGL graph file not found at {dgl_graph_path}")
    sys.exit()
except Exception as e:
    print(f"An error occurred while loading the DGL graph: {e}")
    sys.exit()
# ------------------------------------


# Set dataset_name for output file naming
dataset_name = "sports_outdoors_20k_F" # Explicitly set dataset name for consistent output file naming


# Create results directory if it doesn't exist
if not os.path.exists(args.result_path):
    os.makedirs(args.result_path)

'''
# train model
'''
print('Start training model...')
model = GE_GNN(args, g) # Pass the loaded graph 'g' to the model constructor
model = model.to(device)

optimizer = optim.Adam(params=model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
early_stop = EarlyStop(args.early_stop)

# gradient clip
max_grad_norm = 5.0


for e in range(args.epoch):

    model.train()
    optimizer.zero_grad()
    # Calculate loss using the model's loss method, passing the graph 'g'
    loss = model.loss(g)
    loss.backward()
    # gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()

    with torch.no_grad():
        '''
        # valid
        '''
        model.eval()
        # Use correct mask names for heterogeneous graph
        if 'r' in g.ntypes and 'val_msk' in g.nodes['r'].data:
             valid_mask = g.nodes['r'].data['val_msk'].bool()
             valid_labels = g.nodes['r'].data['label'][valid_mask].cpu().numpy()
             # Corrected: Perform forward pass to get logits for validation
             logits = model(g)[valid_mask]
        elif 'valid_mask' in g.ndata: # Fallback for homogeneous graph
            valid_mask = g.ndata['valid_mask'].bool()
            valid_labels = g.ndata['label'][valid_mask].cpu().numpy()
            # Corrected: Perform forward pass to get logits for validation
            logits = model(g)[valid_mask]
        else:
             print("Warning: 'val_msk' not found in graph. Skipping validation.")
             continue # Skip validation if mask is missing


        if len(valid_labels) == 0:
             print("Warning: No validation samples found. Skipping validation metrics.")
             auc = 0.0 # Assign a default low AUC
             f1_macro, gmean, recall = 0.0, 0.0, 0.0
        else:
            # Corrected: Pass the calculated logits to the evaluate function
            f1_macro, auc, gmean, recall = evaluate(valid_labels, logits)

        if args.log:
            print(f'{e}: Best Epoch:{early_stop.best_epoch}, Best valid AUC:{early_stop.best_eval}, Loss:{loss.item()}, Current valid: Recall:{recall:.4f}, F1_macro:{f1_macro:.4f}, G-Mean:{gmean:.4f}, AUC:{auc:.6f}')

        do_store, do_stop = early_stop.step(auc, e)
        if do_store:
            # Save model state dict
            model_save_path = os.path.join(args.result_path, f"{dataset_name}_model_head{args.head}.pt")
            torch.save(model.state_dict(), model_save_path)
            if args.log:
                 print(f"Saved best model state_dict for {dataset_name} at epoch {e}, best_auc={early_stop.best_eval:.6f}")

        if do_stop:
            break
print('End training')

'''
# test model
'''
print('Test model...')
# Load model state dict
model_load_path = os.path.join(args.result_path, f"{dataset_name}_model_head{args.head}.pt")
if os.path.exists(model_load_path):
    # Create a new model instance before loading state dict
    model = GE_GNN(args, g) # Pass the graph again to initialize correctly
    model.load_state_dict(torch.load(model_load_path, map_location=device))
    model.to(device)
    with torch.no_grad():
        model.eval()
        # Use correct mask names for heterogeneous graph
        if 'r' in g.ntypes and 'tst_msk' in g.nodes['r'].data:
            test_mask = g.nodes['r'].data['tst_msk'].bool()
            test_labels = g.nodes['r'].data['label'][test_mask]
            test_labels = test_labels.cpu().numpy()
            logits = model(g)[test_mask]
            logits = logits.cpu()
        elif 'test_mask' in g.ndata: # Fallback for homogeneous graph
            test_mask = g.ndata['test_mask'].bool()
            test_labels = g.ndata['label'][test_mask]
            test_labels = test_labels.cpu().numpy()
            logits = model(g)[test_mask]
            logits = logits.cpu()
        else:
            print(f"Warning: 'tst_msk' not found in graph for {dataset_name}. Skipping test evaluation.")
            sys.exit() # Exit if test mask is missing


        test_result_path = os.path.join(args.result_path, dataset_name)
        if len(test_labels) == 0:
            print(f"Warning: No test samples found for {dataset_name}. Skipping test metrics.")
        else:
            f1_macro, auc, gmean, recall = evaluate(test_labels, logits, test_result_path)
            print(f'Test: F1-macro:{f1_macro:.6f}, AUC:{auc:.6f}, G-Mean:{gmean:.6f}, Recall:{recall:.6f}')
else:
    print(f"No saved model found at {model_load_path}; skipping test for {dataset_name}.")

# exit()

In [ ]:
# ===========================
# GE-GNN Training (Leak-Safe)
# ===========================

import os, sys, copy, random, warnings, argparse, yaml
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dgl

from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, recall_score

warnings.filterwarnings('ignore')


# ----------------------
# Reproducibility helper
# ----------------------
def setup_seed(seed: int):
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


# ----------------
# EarlyStop helper
# ----------------
class EarlyStop:
    def __init__(self, early_stop: int, if_more: bool = True):
        self.best_eval = -float('inf') if if_more else float('inf')
        self.best_epoch = 0
        self.if_more = if_more
        self.early_stop = early_stop
        self.stop_steps = 0

    def step(self, current_eval: float, current_epoch: int):
        do_stop, do_store = False, False
        better = current_eval > self.best_eval if self.if_more else current_eval < self.best_eval
        if better:
            self.best_eval = current_eval
            self.best_epoch = current_epoch
            self.stop_steps = 0
            do_store = True
        else:
            self.stop_steps += 1
            if self.stop_steps >= self.early_stop:
                do_stop = True
        return do_store, do_stop


# ---------------------
# Metrics / Eval helper
# ---------------------
def _conf_gmean(conf):
    tn, fp, fn, tp = conf.ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    return (tpr * tnr) ** 0.5


def evaluate(labels_np, logits, result_prefix: str = ''):
    probs = F.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()
    preds = logits.argmax(1).detach().cpu().numpy()

    if result_prefix:
        np.save(result_prefix + '_result_preds.npy', preds)
        np.save(result_prefix + '_result_probs.npy', probs)

    if len(np.unique(labels_np)) < 2:
        auc = 0.5
    else:
        auc = roc_auc_score(labels_np, probs)

    conf = confusion_matrix(labels_np, preds, labels=[0, 1])
    recall = recall_score(labels_np, preds, zero_division=0)
    f1_macro = f1_score(labels_np, preds, average='macro', zero_division=0)
    gmean = _conf_gmean(conf)
    return f1_macro, auc, gmean, recall


# ----------------------
# Graph access utilities
# ----------------------
def has_r(g):
    return hasattr(g, 'ntypes') and ('r' in g.ntypes)


def get_feat_key(g):
    """
    Return the node feature key to read from for initial input.
    Preference: 'feature' then 'feat'.
    Works for hetero('r') and homo graphs.
    """
    if has_r(g):
        keys = g.nodes['r'].data.keys()
        if 'feature' in keys:
            return ('r', 'feature')
        if 'feat' in keys:
            return ('r', 'feat')
    else:
        keys = g.ndata.keys()
        if 'feature' in keys:
            return (None, 'feature')
        if 'feat' in keys:
            return (None, 'feat')
    raise KeyError("No node features found. Expected 'feature' or 'feat'.")


def get_label_and_masks(g):
    """
    Return labels tensor and (train, valid, test) masks as boolean tensors.
    Supports both {trn,val,tst}_msk and {train,valid,test}_mask.
    Works for hetero('r') and homo graphs.
    """
    if has_r(g):
        ndata = g.nodes['r'].data
        label_key = 'label'
        if label_key not in ndata:
            raise KeyError("labels not found at nodes['r'].data['label']")

        def pick_mask(prefix_a, prefix_b):
            if prefix_a in ndata:
                return ndata[prefix_a].bool()
            if prefix_b in ndata:
                return ndata[prefix_b].bool()
            return None

        train_mask = pick_mask('trn_msk', 'train_mask')
        valid_mask = pick_mask('val_msk', 'valid_mask')
        test_mask = pick_mask('tst_msk', 'test_mask')

        if train_mask is None or valid_mask is None or test_mask is None:
            raise KeyError("Could not find train/valid/test masks in nodes['r'].data.")

        labels = ndata[label_key].long()
        return labels, train_mask, valid_mask, test_mask
    else:
        ndata = g.ndata
        label_key = 'label'
        if label_key not in ndata:
            raise KeyError("labels not found at g.ndata['label']")

        def pick_mask(prefix_a, prefix_b):
            if prefix_a in ndata:
                return ndata[prefix_a].bool()
            if prefix_b in ndata:
                return ndata[prefix_b].bool()
            return None

        train_mask = pick_mask('trn_msk', 'train_mask')
        valid_mask = pick_mask('val_msk', 'valid_mask')
        test_mask = pick_mask('tst_msk', 'test_mask')

        if train_mask is None or valid_mask is None or test_mask is None:
            raise KeyError("Could not find train/valid/test masks in g.ndata.")

        labels = ndata[label_key].long()
        return labels, train_mask, valid_mask, test_mask


# -------------------------
# GE-GNN Layer primitives
# -------------------------
class RelationAware(nn.Module):
    def __init__(self, input_dim, output_dim, dropout):
        super().__init__()
        self.d_lin = nn.Linear(input_dim, output_dim)
        self.tanh = nn.Tanh()
        self.drop = nn.Dropout(dropout)

    def forward(self, src, dst):
        src = self.d_lin(src)
        dst = self.d_lin(dst)
        diff = src - dst
        return self.tanh(src + dst + diff)


class Gate(nn.Module):
    def __init__(self, head, output_dim, dropout, if_sum=False):
        super().__init__()
        self.head = head
        self.output_dim = output_dim
        if not if_sum:
            self.beta = nn.Parameter(torch.empty(size=(2 * self.head * self.output_dim, 1)))
        else:
            self.beta = nn.Parameter(torch.empty(size=(2 * self.output_dim, 1)))
        nn.init.xavier_normal_(self.beta.data, gain=1.414)
        self.sigmoid = nn.Sigmoid()

    def forward(self, edge_sum, out, h):
        beta = torch.cat([edge_sum, out], dim=1)
        gate = self.sigmoid(beta @ self.beta)
        return gate * out + (1 - gate) * h


class HLayer(nn.Module):
    """
    One relation head.
    Works for both hetero and homo graphs.
    """
    def __init__(self, input_dim, output_dim, head, relation_aware, etype, dropout, if_sum=False):
        super().__init__()
        self.etype = etype        # None for homogeneous
        self.head = head
        self.hd = output_dim
        self.if_sum = if_sum
        self.relation_aware = relation_aware

        self.atten = nn.Linear(3 * self.hd, 1)
        self.relu = nn.ReLU()
        self.leakyrelu = nn.LeakyReLU()
        self.softmax = nn.Softmax(dim=1)
        self.w_linear = nn.Linear(input_dim, output_dim * head)

    def _apply_edges(self, g):
        # message fn needs node field 'h_in' present
        def sign_edges(edges):
            src = edges.src['h_in']
            dst = edges.dst['h_in']
            edge_sum = self.relation_aware(src, dst)  # shape [E, output_dim*head]
            return {'edge_sum': edge_sum}
        return sign_edges

    def message(self, edges):
        src_f = edges.src['h']
        dst_f = edges.dst['h']
        src_f = src_f.view(-1, self.head, self.hd)
        dst_f = dst_f.view(-1, self.head, self.hd)
        edge_s = edges.data['edge_sum'].view(-1, self.head, self.hd)
        z = torch.cat([src_f, dst_f, edge_s], dim=-1)
        alpha = self.leakyrelu(self.atten(z))
        return {'atten': alpha, 'sf': src_f, 'edge_s': edge_s}

    def reduce(self, nodes):
        alpha = self.softmax(nodes.mailbox['atten'])
        sf = nodes.mailbox['sf']
        out = torch.sum(alpha * sf, dim=1)
        if not self.if_sum:
            out = out.view(-1, self.head * self.hd)
            edge_s = torch.mean(nodes.mailbox['edge_s'], dim=1).view(-1, self.head * self.hd)
            return {'out': out, 's': edge_s}
        else:
            out = out.sum(dim=-2)
            edge_s = torch.sum(torch.mean(nodes.mailbox['edge_s'], dim=1), dim=-2)
            return {'out': out, 's': edge_s}

    def forward(self, g, h):
        with g.local_scope():
            # project features for messages
            h_proj = self.w_linear(h)

            # set working fields
            if has_r(g):
                g.nodes['r'].data['h_in'] = h          # for edge signing
                g.nodes['r'].data['h'] = h_proj        # for message passing
                if self.etype is None:
                    # shouldn't happen for hetero; but keep guard
                    g.apply_edges(self._apply_edges(g))
                    g.update_all(self.message, self.reduce)
                    out = g.nodes['r'].data['out']
                    edge_s = g.nodes['r'].data['s']
                else:
                    g.apply_edges(self._apply_edges(g), etype=self.etype)
                    g.update_all(self.message, self.reduce, etype=self.etype)
                    out = g.nodes['r'].data['out']
                    edge_s = g.nodes['r'].data['s']
            else:
                g.ndata['h_in'] = h
                g.ndata['h'] = h_proj
                g.apply_edges(self._apply_edges(g))
                g.update_all(self.message, self.reduce)
                out = g.ndata['out']
                edge_s = g.ndata['s']

            if not self.if_sum:
                return edge_s, out, h_proj.view(-1, self.head * self.hd)
            else:
                return edge_s, out, h_proj.view(-1, self.head, self.hd).sum(-2)


class MultiRelationGE_GNNLayer(nn.Module):
    def __init__(self, input_dim, output_dim, head, graph_for_etypes, dropout, if_sum=False):
        super().__init__()
        # collect relations
        etypes = list(graph_for_etypes.etypes) if hasattr(graph_for_etypes, 'etypes') else []
        # For homogeneous graphs, etypes could be ['_E'] or empty; treat as single relation None
        self.relations = etypes if (has_r(graph_for_etypes) and len(etypes) > 0) else [None]
        self.if_sum = if_sum
        self.n_rel = len(self.relations)

        self.lin = nn.Linear(self.n_rel * (output_dim if if_sum else output_dim * head),
                             (output_dim if if_sum else output_dim * head))

        self.rel_aware = RelationAware(input_dim, output_dim * head, dropout)
        self.blocks = nn.ModuleDict()
        for e in self.relations:
            block = nn.ModuleList([
                HLayer(input_dim, output_dim, head, self.rel_aware, e, dropout, if_sum),
                Gate(head, output_dim, dropout, if_sum)
            ])
            self.blocks[str(e)] = block

        self.drop = nn.Dropout(dropout)

    def forward(self, g, h):
        hs = []
        for e in self.relations:
            edge_sum, out, h_lin = self.blocks[str(e)][0](g, h)
            he = self.blocks[str(e)][1](edge_sum, out, h_lin)
            hs.append(he)
        x = torch.cat(hs, dim=1) if len(hs) > 1 else hs[0]
        x = self.drop(x)
        return self.lin(x)


# -------------
# GE-GNN model
# -------------
class GE_GNN(nn.Module):
    def __init__(self, args, g, n_class=2):
        super().__init__()
        # input dim
        ntype, feat_key = get_feat_key(g)
        if has_r(g):
            in_dim = g.nodes['r'].data[feat_key].shape[1] if ntype == 'r' else g.ndata[feat_key].shape[1]
        else:
            in_dim = g.ndata[feat_key].shape[1]

        self.n_layer = args.n_layer
        self.intra_dim = args.intra_dim
        self.head = args.head
        self.n_class = n_class
        self.drop = nn.Dropout(args.dropout)
        self.relu = nn.ReLU()

        layers = []
        if self.n_layer == 1:
            layers.append(MultiRelationGE_GNNLayer(in_dim, self.n_class, self.head, g, args.dropout, if_sum=True))
        else:
            layers.append(MultiRelationGE_GNNLayer(in_dim, self.intra_dim, self.head, g, args.dropout, if_sum=False))
            for _ in range(1, self.n_layer - 1):
                layers.append(MultiRelationGE_GNNLayer(self.intra_dim * self.head, self.intra_dim, self.head, g, args.dropout, if_sum=False))
            layers.append(MultiRelationGE_GNNLayer(self.intra_dim * self.head, self.n_class, self.head, g, args.dropout, if_sum=True))
        self.layers = nn.ModuleList(layers)

    def _read_init_feat(self, g):
        ntype, feat_key = get_feat_key(g)
        if has_r(g) and ntype == 'r':
            return g.nodes['r'].data[feat_key].float()
        elif not has_r(g):
            return g.ndata[feat_key].float()
        else:
            raise RuntimeError("Cannot find initial features for forward().")

    def forward(self, g):
        h = self._read_init_feat(g)
        for i, layer in enumerate(self.layers):
            h = layer(g, h)
            if i < len(self.layers) - 1:
                h = self.relu(h)
                h = self.drop(h)
        return h  # logits


# ---------------
# Config / Paths
# ---------------
# If you prefer YAML: set config_path and load. Otherwise hardcode here.
config_path = "/content/drive/MyDrive/GE-GNN/config/amazon.yaml"
with open(config_path, 'r') as f:
    cfg = yaml.safe_load(f)

# You can override unstable params here if needed:
cfg.setdefault('lr', 0.003)               # a bit lower than 0.01 for stability
cfg.setdefault('weight_decay', 5e-4)
cfg.setdefault('epoch', 300)
cfg.setdefault('early_stop', 50)
cfg.setdefault('dropout', 0.3)
cfg.setdefault('head', 4)
cfg.setdefault('n_layer', 3)
cfg.setdefault('intra_dim', 64)
cfg.setdefault('n_class', 2)
cfg.setdefault('seed', 42)
cfg.setdefault('cuda', 0)
cfg.setdefault('result_path', "/content/drive/MyDrive/GE-GNN/result")

args = argparse.Namespace(**cfg)

setup_seed(args.seed)
device = torch.device(f'cuda:{args.cuda}' if torch.cuda.is_available() else 'cpu')
args.device = device

dgl_graph_path = "/content/drive/MyDrive/GE-GNN/Mat_Dgl_files/sports_outdoors_hetero_pavan_20k_F.dgl"
dataset_name = "sports_outdoors_20k_F"

# -------------
# Load the graph
# -------------
try:
    graphs, _ = dgl.load_graphs(dgl_graph_path)
    g = graphs[0]
    print(f"Loaded DGL graph from: {dgl_graph_path}")
except Exception as e:
    print(f"Failed to load graph: {e}")
    sys.exit(1)

# Move to device & ensure node data on device
g = g.to(device)
if has_r(g):
    for k in list(g.nodes['r'].data.keys()):
        g.nodes['r'].data[k] = g.nodes['r'].data[k].to(device)
else:
    for k in list(g.ndata.keys()):
        g.ndata[k] = g.ndata[k].to(device)

# --------
# Training
# --------
os.makedirs(args.result_path, exist_ok=True)

labels, train_mask, valid_mask, test_mask = get_label_and_masks(g)
labels_np_valid = labels[valid_mask].detach().cpu().numpy()

print('Start training model...')
model = GE_GNN(args, g, n_class=args.n_class).to(device)

# Class weights (optional but helpful if class-imbalance)
with torch.no_grad():
    y_train = labels[train_mask].detach().cpu().numpy()
    # avoid div by zero; if balanced, weights ~1
    pos = (y_train == 1).sum()
    neg = (y_train == 0).sum()
    if pos > 0 and neg > 0:
        w0 = 0.5 * (pos + neg) / neg
        w1 = 0.5 * (pos + neg) / pos
        class_weight = torch.tensor([w0, w1], dtype=torch.float32, device=device)
    else:
        class_weight = None

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
early_stop = EarlyStop(args.early_stop, if_more=True)
max_grad_norm = 5.0

best_model_path = os.path.join(args.result_path, f"{dataset_name}_model_head{args.head}.pt")

for e in range(args.epoch):
    model.train()
    optimizer.zero_grad()

    logits = model(g)
    if train_mask.sum() == 0:
        print("No training nodes in mask; aborting.")
        break
    loss = F.cross_entropy(logits[train_mask], labels[train_mask], weight=class_weight)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()

    # VALID
    model.eval()
    with torch.no_grad():
        if valid_mask.sum() > 0:
            val_logits = logits[valid_mask]  # reuse forward result for speed
            f1_macro, auc, gmean, recall = evaluate(labels_np_valid, val_logits)
        else:
            f1_macro, auc, gmean, recall = (0.0, 0.0, 0.0, 0.0)

    print(f"{e}: Best Epoch:{early_stop.best_epoch}, Best valid AUC:{early_stop.best_eval:.6f}, "
          f"Loss:{loss.item():.6f}, Current valid: Recall:{recall:.4f}, "
          f"F1_macro:{f1_macro:.4f}, G-Mean:{gmean:.4f}, AUC:{auc:.6f}")

    do_store, do_stop = early_stop.step(auc, e)
    if do_store:
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved best model state_dict for {dataset_name} at epoch {e}, best_auc={early_stop.best_eval:.6f}")
    if do_stop:
        print(f"Early stopping at epoch {e}. Best epoch: {early_stop.best_epoch}")
        break

print('End training')

# ----
# Test
# ----
print('Test model...')
if os.path.exists(best_model_path):
    model = GE_GNN(args, g, n_class=args.n_class).to(device)
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.eval()

    with torch.no_grad():
        logits = model(g)
        if test_mask.sum() == 0:
            print("No test nodes found; skipping test.")
        else:
            test_logits = logits[test_mask]
            test_labels_np = labels[test_mask].detach().cpu().numpy()
            test_prefix = os.path.join(args.result_path, dataset_name)
            f1_macro, auc, gmean, recall = evaluate(test_labels_np, test_logits, test_prefix)
            print(f"Test: F1-macro:{f1_macro:.6f}, AUC:{auc:.6f}, G-Mean:{gmean:.6f}, Recall:{recall:.6f}")
else:
    print(f"No saved model found at {best_model_path}; skipping test.")
